In [ ]:
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.functional as f 
from torch.autograd import Variable
torch.backends.cudnn.deterministic = True
import pandas as pd
import numpy as np
import torch.utils.data as data_utils
from sklearn.preprocessing import StandardScaler 
from sklearn.model_selection import train_test_split
from sklearn import metrics
import random 
from matplotlib import pyplot as plt
from sklearn import decomposition
from sklearn import manifold
from scipy import stats
from itertools import cycle
import sys
import utilities as ut
from Network import Net
import Network as nn
SEED = 1234
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
import matplotlib

matplotlib.rcParams["figure.dpi"] = 100


use_cuda = torch.cuda.is_available()
device = torch.device("cuda:0" if use_cuda else "cpu")
torch.backends.cudnn.benchmark = True

#ab y c con 3000 samples, 256 batch y 10000 objetos. 

results = []
num_classes = 2
learning_rate = 0.001
samples = 3000 #
epsilon=0
#for epsilon in [0.1, 0.05, 0.025, 0.15]:
for sc in ['_RRab', '_RRc']: #,'_RRc','_RRab'
    for batch_size in [256]:
        for hidden_size in [100]:
            for aux_loss_activated in [False]:
                for EPS1 in [0.005]:
                    for n in [5000, 10000]:
                        for opt in [1]:
                            for t in range(15):
                                train_dataset, test_dataset = ut.load_files(dataset=1, subclass=sc)
                                input_size = train_dataset.shape[1]-1

                                if n < 50000:
                                    train_dataset = ut.down_sampling(train_dataset)
                                    train_dataset = train_dataset.sample(n)
                                    print(train_dataset)
                                else: 
                                    trainig_dataset_a = train_dataset[train_dataset.label=='ClassA']
                                    print('shape: ', trainig_dataset_a.shape[0])
                                    n2 = n - trainig_dataset_a.shape[0]
                                    print('clase no RR Lrae', n2)
                                    trainig_dataset_b = train_dataset[~(train_dataset.label=='ClassA')].sample(n2)
                                    train_dataset = pd.concat([trainig_dataset_a, trainig_dataset_b])

                                train_dataset, _ = ut.delete_outliers(train_dataset, test_dataset)

                                train_dataset = ut.sort_columns(train_dataset)
                                test_dataset = ut.sort_columns(test_dataset)

                                test_dataset_pred = test_dataset.copy()
                                train_dataset_pred = train_dataset.copy()

                                try:
                                    data_prior = ut.generate_samples_2D(samples, train_dataset, distribution='uniform')

                                    train_dataset, val_dataset = train_test_split(train_dataset, test_size=0.2, random_state=42)

                                    train_dataset_prior, val_dataset_prior = train_test_split(data_prior, test_size=0.2, random_state=42)
                                    print(train_dataset_prior.columns)
                                    _, _, train_target_prior, train_loader_prior = ut.get_tensors(train_dataset_prior, batch_size)
                                    _, _, val_target_prior, val_loader_prior     = ut.get_tensors(val_dataset_prior, batch_size)
                                    _, _, train_target, train_loader             = ut.get_tensors(train_dataset, batch_size)
                                    _, _, train_target_pred, train_loader_pred   = ut.get_tensors(train_dataset_pred, batch_size)
                                    _, _, val_target, val_loader                 = ut.get_tensors(val_dataset_prior, batch_size)
                                    _, _, test_target, test_loader               = ut.get_tensors(test_dataset, batch_size)
                                    _, _, test_target_pred, test_loader_pred     = ut.get_tensors(test_dataset_pred, batch_size)

                                    net = Net(input_size, hidden_size, hidden_size, num_classes)
                                    net.cuda()

                                    hist_val, hist_train = nn.train(net, train_loader, train_loader_prior, val_loader, test_loader,
                                    EPS1, learning_rate, input_size, aux_loss_activated=aux_loss_activated, patience=10)

                                    acc_train, recall_train, f1_train = nn.get_results(net, train_loader, input_size)
                                    acc_test, recall_test, f1_test  = nn.get_results(net, test_loader, input_size)
                                    results.append([acc_train, acc_test,recall_train, recall_test, f1_train, f1_test, epsilon, batch_size, hidden_size, aux_loss_activated, EPS1, n, opt, sc])
                                    pd.DataFrame(results, columns=['acc_train', 'acc_test','recall_train', 'recall_test','f1_train', 'f1_test',
                                     'epsilon', 'batch_size', 'hidden_size',
                                     'aux_loss_activated', 'EPS1', 'n', 'opt', 'sc']).to_csv('subclasses_07-05-2022-baseline.csv')
                                except Exception as e:
                                    print(e) 
                                    print(str(batch_size)+"-"+str(hidden_size)+"-"+str(aux_loss_activated)+"-"+str(EPS1))

/home/franciscoperez/anaconda3/lib/python3.7/site-packages/pandas/compat/_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.8' currently installed).
  warnings.warn(msg, UserWarning)
In /home/franciscoperez/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/franciscoperez/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In /home/franciscoperez/anaconda3/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In /home/franciscoperez/anacon

Freq1_harmonics_rel_phase_0
Freq2_harmonics_rel_phase_0
Freq3_harmonics_rel_phase_0
downsampling
21885
369616
done downsampling
        Amplitude  AndersonDarling  Autocor_length  Beyond1Std    CAR_mean  \
5472       0.2755         0.229660               1    0.353333   23.520652   
59067      0.3450         0.000116               1    0.370000   55.028761   
325888     0.2405         0.003151               1    0.353333   26.861221   
315033     0.0540         0.000000               1    0.173333  193.960646   
67839      0.1390         0.000000              11    0.206667    0.071036   
...           ...              ...             ...         ...         ...   
88103      0.0495         0.060294               4    0.338983   58.234361   
85233      0.1010         0.000000               7    0.220000   32.254755   
342154     0.0185         0.147566               2    0.333333   15.465235   
95854      0.5475         0.000000              13    0.107023    0.010237   
221400     0.3

/home/franciscoperez/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


Epochs:  1000
Epoch:  0
training: epoch:  1  loss:  8.041600167751312 -- aux loss:  0.0
validating: epoch:  1  loss:  3.4558692574501038
Epoch:  1
training: epoch:  2  loss:  6.998945236206055 -- aux loss:  0.0
validating: epoch:  2  loss:  3.4191596508026123
Epoch:  2
training: epoch:  3  loss:  6.357063055038452 -- aux loss:  0.0
validating: epoch:  3  loss:  3.382687747478485
Epoch:  3
training: epoch:  4  loss:  6.089393079280853 -- aux loss:  0.0
validating: epoch:  4  loss:  3.3365578651428223
Epoch:  4
training: epoch:  5  loss:  5.922102242708206 -- aux loss:  0.0
validating: epoch:  5  loss:  3.285922348499298
Epoch:  5
training: epoch:  6  loss:  5.972275584936142 -- aux loss:  0.0
validating: epoch:  6  loss:  3.2238935232162476
Epoch:  6
training: epoch:  7  loss:  5.691137462854385 -- aux loss:  0.0
validating: epoch:  7  loss:  3.1678683161735535
Epoch:  7
training: epoch:  8  loss:  5.295112371444702 -- aux loss:  0.0
validating: epoch:  8  loss:  3.1030492186546326
Epoc

validating: epoch:  44  loss:  1.6906971335411072
Epoch:  44
training: epoch:  45  loss:  4.835298657417297 -- aux loss:  0.0
validating: epoch:  45  loss:  1.6894988715648651
Epoch:  45
training: epoch:  46  loss:  4.826413869857788 -- aux loss:  0.0
validating: epoch:  46  loss:  1.6826004385948181
Epoch:  46
training: epoch:  47  loss:  4.7020997405052185 -- aux loss:  0.0
validating: epoch:  47  loss:  1.6779928505420685
Epoch:  47
training: epoch:  48  loss:  4.6065521240234375 -- aux loss:  0.0
validating: epoch:  48  loss:  1.6722558438777924
Epoch:  48
training: epoch:  49  loss:  4.51946547627449 -- aux loss:  0.0
validating: epoch:  49  loss:  1.666439801454544
Epoch:  49
training: epoch:  50  loss:  4.426755726337433 -- aux loss:  0.0
validating: epoch:  50  loss:  1.6609622240066528
The current loss: 1.6609622240066528
the_last_loss: 1.666439801454544
trigger times: 0
recall
tensor(0.9941)
precision
tensor(0.9658)
f1_score
tensor(0.9797)
Accuracy of the network on test obje

...

training: epoch:  820  loss:  4.306986093521118 -- aux loss:  0.0
validating: epoch:  820  loss:  1.5663127303123474
The current loss: 1.5663127303123474
the_last_loss: 1.5663127899169922
trigger times: 0
recall
tensor(0.9989)
precision
tensor(0.9706)
f1_score
tensor(0.9845)
Accuracy of the network on test objects: 97 %
97.057274
recall
tensor(1.)
precision
tensor(1.)
f1_score
tensor(1.)
Accuracy of the network on test objects: 100 %
100.0
recall
tensor(0.9473)
precision
tensor(0.3604)
f1_score
tensor(0.5221)
Accuracy of the network on test objects: 36 %
36.03872
sum mask2 - L1:  tensor(5784, device='cuda:0')
sum mask2 - L2:  tensor(9475, device='cuda:0')
sum mask2 - L3:  tensor(191, device='cuda:0')
sum mask1 - L1 (aux):  tensor(216, device='cuda:0')
sum mask1 - L2 (aux):  tensor(525, device='cuda:0')
sum mask1 - L3 (aux):  tensor(9, device='cuda:0')
Epoch:  820
training: epoch:  821  loss:  4.306986033916473 -- aux loss:  0.0
validating: epoch:  821  loss:  1.566312700510025
Epoch:  

tensor(1.)
f1_score
tensor(1.)
Accuracy of the network on test objects: 100 %
100.0
recall
tensor(0.9474)
precision
tensor(0.3605)
f1_score
tensor(0.5222)
Accuracy of the network on test objects: 36 %
36.04673
sum mask2 - L1:  tensor(5784, device='cuda:0')
sum mask2 - L2:  tensor(9475, device='cuda:0')
sum mask2 - L3:  tensor(191, device='cuda:0')
sum mask1 - L1 (aux):  tensor(216, device='cuda:0')
sum mask1 - L2 (aux):  tensor(525, device='cuda:0')
sum mask1 - L3 (aux):  tensor(9, device='cuda:0')
Epoch:  860
training: epoch:  861  loss:  4.306981474161148 -- aux loss:  0.0
validating: epoch:  861  loss:  1.5663110315799713
Epoch:  861
training: epoch:  862  loss:  4.306981295347214 -- aux loss:  0.0
validating: epoch:  862  loss:  1.566311001777649
Epoch:  862
training: epoch:  863  loss:  4.306981146335602 -- aux loss:  0.0
validating: epoch:  863  loss:  1.5663108825683594
Epoch:  863
training: epoch:  864  loss:  4.306981086730957 -- aux loss:  0.0
validating: epoch:  864  loss:  

recall
tensor(0.9474)
precision
tensor(0.3604)
f1_score
tensor(0.5222)
Accuracy of the network on test objects: 36 %
36.043373
sum mask2 - L1:  tensor(5784, device='cuda:0')
sum mask2 - L2:  tensor(9475, device='cuda:0')
sum mask2 - L3:  tensor(191, device='cuda:0')
sum mask1 - L1 (aux):  tensor(216, device='cuda:0')
sum mask1 - L2 (aux):  tensor(525, device='cuda:0')
sum mask1 - L3 (aux):  tensor(9, device='cuda:0')
Epoch:  900
training: epoch:  901  loss:  4.3069775104522705 -- aux loss:  0.0
validating: epoch:  901  loss:  1.566310077905655
Epoch:  901
training: epoch:  902  loss:  4.306977450847626 -- aux loss:  0.0
validating: epoch:  902  loss:  1.566310077905655
Epoch:  902
training: epoch:  903  loss:  4.306977391242981 -- aux loss:  0.0
validating: epoch:  903  loss:  1.566310077905655
Epoch:  903
training: epoch:  904  loss:  4.306977272033691 -- aux loss:  0.0
validating: epoch:  904  loss:  1.566310077905655
Epoch:  904
training: epoch:  905  loss:  4.306977182626724 -- aux

training: epoch:  941  loss:  4.306974291801453 -- aux loss:  0.0
validating: epoch:  941  loss:  1.5663097202777863
Epoch:  941
training: epoch:  942  loss:  4.3069742023944855 -- aux loss:  0.0
validating: epoch:  942  loss:  1.5663097202777863
Epoch:  942
training: epoch:  943  loss:  4.306974112987518 -- aux loss:  0.0
validating: epoch:  943  loss:  1.5663097202777863
Epoch:  943
training: epoch:  944  loss:  4.306974083185196 -- aux loss:  0.0
validating: epoch:  944  loss:  1.5663096904754639
Epoch:  944
training: epoch:  945  loss:  4.306974023580551 -- aux loss:  0.0
validating: epoch:  945  loss:  1.5663096904754639
Epoch:  945
training: epoch:  946  loss:  4.306973993778229 -- aux loss:  0.0
validating: epoch:  946  loss:  1.5663096904754639
Epoch:  946
training: epoch:  947  loss:  4.306973844766617 -- aux loss:  0.0
validating: epoch:  947  loss:  1.5663096904754639
Epoch:  947
training: epoch:  948  loss:  4.306973814964294 -- aux loss:  0.0
validating: epoch:  948  loss:

training: epoch:  986  loss:  4.306971222162247 -- aux loss:  0.0
validating: epoch:  986  loss:  1.5663093328475952
Epoch:  986
training: epoch:  987  loss:  4.3069711327552795 -- aux loss:  0.0
validating: epoch:  987  loss:  1.5663093328475952
Epoch:  987
training: epoch:  988  loss:  4.30697101354599 -- aux loss:  0.0
validating: epoch:  988  loss:  1.5663093328475952
Epoch:  988
training: epoch:  989  loss:  4.306970983743668 -- aux loss:  0.0
validating: epoch:  989  loss:  1.5663093328475952
Epoch:  989
training: epoch:  990  loss:  4.306970983743668 -- aux loss:  0.0
validating: epoch:  990  loss:  1.5663093328475952
The current loss: 1.5663093328475952
the_last_loss: 1.5663093328475952
trigger times: 0
recall
tensor(0.9989)
precision
tensor(0.9706)
f1_score
tensor(0.9845)
Accuracy of the network on test objects: 97 %
97.057274
recall
tensor(1.)
precision
tensor(1.)
f1_score
tensor(1.)
Accuracy of the network on test objects: 100 %
100.0
recall
tensor(0.9476)
precision
tensor(0

In [ ]:
nn.get_results(net, train_loader, input_size)
nn.get_results(net, test_loader, input_size)

In [ ]:
nn.get_roc_curve(net, test_loader, input_size)

In [ ]:
torch.save(net.state_dict(), 'model.pkl')

In [ ]:
ut.plot_training(hist_val, hist_train)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(6,6))
curves, labels, probs_test = ut.get_predictions(net, test_loader_pred, device)
pred_labels = probs_test.argmax(1, keepdim = True)
ut.plot_confusion_matrix(np.round(labels), pred_labels, ax)

In [ ]:
#csv_file = open("size_MLP_noise.csv", "a")
#csv_file.write(str(np.asarray(acc_testing))+","+str(np.asarray(acc_training))+","+str(samples)+","+str(epsilon)+","+str(n)+","+str(hidden_size)+"\n")
#csv_file.close()

In [ ]:
outputs, intermediates, intermediates2, labels = ut.get_representations(net, train_loader, device)
outputs_test, intermediates_test, intermediates2_test, labels_test = ut.get_representations(net, test_loader, device)

In [ ]:
intermediate_pca_data, intermediate_pca_data_test = ut.get_pca(intermediates, data_test=intermediates_test)
fig, axs = plt.subplots(2, 3, figsize=(20,15))
ut.plot_representations(intermediate_pca_data, labels, axs[0, 0])
ut.plot_representations(intermediate_pca_data_test, labels_test, axs[1, 0])
intermediate2_pca_data, intermediate2_pca_data_test = ut.get_pca(intermediates2, data_test=intermediates2_test)
ut.plot_representations(intermediate2_pca_data, labels, axs[0, 1])
ut.plot_representations(intermediate2_pca_data_test, labels_test, axs[1, 1])
output_pca_data, output_pca_data_test = ut.get_pca(outputs, data_test=outputs_test)
ut.plot_representations(output_pca_data, labels, axs[0, 2])
ut.plot_representations(output_pca_data_test, labels_test, axs[1, 2])

In [ ]:
import umap

fig, axs = plt.subplots(2, 3, figsize=(20,15))
print('first')
reducer = umap.UMAP()
print('first ..')
mapper_train = reducer.fit_transform(intermediates.numpy())
print('first ...')
reducer = umap.UMAP()
mapper_test = reducer.fit_transform(intermediates_test.numpy())
ut.plot_representations(mapper_train, labels, axs[0, 0])
ut.plot_representations(mapper_test, labels_test, axs[1, 0])

reducer = umap.UMAP()
mapper_train = reducer.fit_transform(intermediates2.numpy())
reducer = umap.UMAP()
mapper_test = reducer.fit_transform(intermediates2_test.numpy())
ut.plot_representations(mapper_train, labels, axs[0, 1])
ut.plot_representations(mapper_test, labels_test, axs[1, 1])

reducer = umap.UMAP()
mapper_train = reducer.fit_transform(outputs.numpy())
reducer = umap.UMAP()
mapper_test = reducer.fit_transform(outputs_test.numpy())

ut.plot_representations(mapper_train, labels, axs[0, 2])
ut.plot_representations(mapper_test, labels_test, axs[1, 2])

In [ ]:
N_CURVES = 25000

fig, axs = plt.subplots(2, 3, figsize=(20,15))
intermediate_tsne_data, intermediate_tsne_data_test = ut.get_tsne(intermediates, data_test= intermediates_test, n_curves = N_CURVES)
ut.plot_representations(intermediate_tsne_data, labels, axs[0, 0],  n_curves = N_CURVES)
ut.plot_representations(intermediate_tsne_data_test, labels_test, axs[1, 0], n_curves = N_CURVES)

intermediate2_tsne_data, intermediate2_tsne_data_test = ut.get_tsne(intermediates2, data_test=intermediates2_test, n_curves = N_CURVES)
ut.plot_representations(intermediate2_tsne_data, labels, axs[0, 1], n_curves = N_CURVES)
ut.plot_representations(intermediate2_tsne_data_test, labels_test, axs[1, 1], n_curves = N_CURVES)

output_tsne_data, output2_tsne_data_test = ut.get_tsne(outputs, data_test=outputs_test, n_curves = N_CURVES)
ut.plot_representations(output_tsne_data, labels, axs[0, 2], n_curves = N_CURVES)
ut.plot_representations(output2_tsne_data_test, labels_test, axs[1, 2], n_curves = N_CURVES)

In [ ]:
#fig, ax = plt.subplots(1, 1, figsize=(15,15))
#curves, labels, probs_train = ut.get_predictions(net, train_loader_pred, device)
#pred_labels = probs_train.argmax(1, keepdim = True)
#ut.plot_confusion_matrix(np.round(labels), pred_labels, ax)


In [ ]:
pred_labels

In [ ]:
curves, labels, probs_train_sample = ut.get_predictions(net, train_loader, device)

In [ ]:
#N_WEIGHTS = 25
#weights = net.fc2.weight.data
#plot_weights(weights, N_WEIGHTS)

In [ ]:
weights1 = net.fc1.weight.data
w1 = weights1.cpu().numpy().reshape(-1,1)
weights2 = net.fc2.weight.data
w2 = weights2.cpu().numpy().reshape(-1,1)
weights3 = net.fc3.weight.data
w3 = weights3.cpu().numpy().reshape(-1,1)

In [ ]:
fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20,15))
ax1.hist(w1, color='black')
ax1.set_xlabel('Layer 1')
ax2.hist(w2, color='black')
ax2.set_xlabel('Layer 2')
ax3.hist(w3, color='black')
ax3.set_xlabel('Layer 3')
plt.show()

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(15,15))
curves, labels, probs_train = ut.get_predictions(net, train_loader_pred, device)
pred_labels = torch.argmax(probs_train, 1)
ut.plot_confusion_matrix(np.round(labels), pred_labels, ax1)
curves, labels, probs_test = ut.get_predictions(net, test_loader_pred, device)
pred_labels = torch.argmax(probs_test, 1)
ut.plot_confusion_matrix(np.round(labels), pred_labels, ax2)